# R Serving with Plumber

## Dockerfile

* The Dockerfile defines the environment in which our server will be executed.
* Below, you can see that the entrypoint for our container will be [deploy.R](deploy.R)

In [1]:
%pycat Dockerfile

FROM r-base:3.6.3

MAINTAINER Amazon SageMaker Examples <amazon-sagemaker-examples@amazon.com>

RUN apt-get -y update && apt-get install -y --no-install-recommends \
    wget \
    apt-transport-https \
    ca-certificates \
    libcurl4-openssl-dev \
    libsodium-dev

RUN R -e "install.packages(c('xgboost','plumber'), repos='https://cloud.r-project.org')"

COPY xgb.model /opt/ml/xgb.model
COPY endpoints.R /opt/ml/endpoints.R
COPY deploy.R /opt/ml/deploy.R

WORKDIR /opt/ml

ENTRYPOINT ["/usr/bin/Rscript", "/opt/ml/deploy.R", "--no-save"]


## Code: deploy.R

The **deploy.R** script handles the following steps:
* Loads the R libraries used by the server.
* Loads a pretrained `xgboost` model that has been trained on the classical [Iris](https://archive.ics.uci.edu/ml/datasets/iris) dataset.
  * Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.
* Defines an inference function that takes a matrix of iris features and returns predictions for those iris examples.
* Finally, it imports the [endpoints.R](endpoints.R) script and launches the Plumber server app using those endpoint definitions.


In [2]:
%pycat deploy.R

library(xgboost)
library(plumber)
library(jsonlite)

# load a pretrained xgboost model
bst <- xgb.load("xgb.model")

# create a closure around our xgboost model and input data processing
inference <- function(x){
  ds <- xgb.DMatrix(data = x )
  output <- predict(bst, ds)
  list(output=output)
}

app <- plumb('endpoints.R')
app$run(host='0.0.0.0', port=8080)


## Code: endpoints.R

**endpoints.R** defines two routes:
* `/ping` returns a string 'Alive' to indicate that the application is healthy
* `/invocations` applies the previously defined inference function to the input features from the request body

For more information about the requirements for building your own inference container, see:
[Use Your Own Inference Code with Hosting Services](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html)

In [ ]:
%pycat endpoints.R

## Build the Serving Image

In [3]:
!docker build -t r-plumber .

Sending build context to Docker daemon  114.2kB
Step 1/9 : FROM r-base:3.6.3
3.6.3: Pulling from library/r-base

a903b83a: Pulling fs layer 
fdbe572d: Pulling fs layer 
e05835c5: Pulling fs layer 
b43f1097: Pulling fs layer 
12da32ab: Pulling fs layer 
Digest: sha256:3a3fb1e083cbd74c4f3ad43c79ed5d7332301dc573b565e47a8c1decef9bdf36[6A
Status: Downloaded newer image for r-base:3.6.3
 ---> cec2502269fb
Step 2/9 : MAINTAINER Amazon SageMaker Examples <amazon-sagemaker-examples@amazon.com>
 ---> Running in eea5cc986bbc
Removing intermediate container eea5cc986bbc
 ---> 7abd0c49dbbc
Step 3/9 : RUN apt-get -y update && apt-get install -y --no-install-recommends     wget     apt-transport-https     ca-certificates     libcurl4-openssl-dev     libsodium-dev
 ---> Running in f8071f7f1599
Get:1 http://deb.debian.org/debian testing InRelease [112 kB]
Get:2 http://deb.debian.org/debian testing/main amd64 Packages [8,339 kB]
Get:3 http://cdn-fastly.deb.debian.org/debian sid InRelease [165 kB]
Get:4 

## Launch the Serving Container

In [4]:
!echo "Launching Plumber"
!docker run -d --rm -p 5000:8080 r-plumber
!echo "Waiting for the server to start.." && sleep 10

Launching Plumber
0edbddf74fdc68c14b1dd92c910f9a1ec1bff8f6d64a100906f6ad237f8943f2
Waiting for the server to start..


In [5]:
!docker container list

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                    NAMES
0edbddf74fdc        r-plumber           "/usr/bin/Rscript /o…"   11 seconds ago      Up 10 seconds       0.0.0.0:5000->8080/tcp   festive_perlman


## Define Simple Python Client

In [6]:
import requests
from tqdm import tqdm
import pandas as pd

pd.set_option("display.max_rows", 500)

In [7]:
def get_predictions(examples, instance=requests, port=5000):
    payload = {"features": examples}
    return instance.post(f"http://127.0.0.1:{port}/invocations", json=payload)

In [8]:
def get_health(instance=requests, port=5000):
    instance.get(f"http://127.0.0.1:{port}/ping")

## Define Example Inputs

Let's define example inputs from the Iris dataset.

In [9]:
column_names = ["Sepal.Length", "Sepal.Width", "Petal.Length", "Petal.Width", "Label"]
iris = pd.read_csv(
    "s3://sagemaker-sample-files/datasets/tabular/iris/iris.data", names=column_names
)

In [10]:
iris_features = iris[["Sepal.Length", "Sepal.Width", "Petal.Length", "Petal.Width"]]

In [11]:
example_inputs = iris_features.values.tolist()

### Plumber

In [16]:
example_inputs

[[5.1, 3.5, 1.4, 0.2],
 [4.9, 3.0, 1.4, 0.2],
 [4.7, 3.2, 1.3, 0.2],
 [4.6, 3.1, 1.5, 0.2],
 [5.0, 3.6, 1.4, 0.2],
 [5.4, 3.9, 1.7, 0.4],
 [4.6, 3.4, 1.4, 0.3],
 [5.0, 3.4, 1.5, 0.2],
 [4.4, 2.9, 1.4, 0.2],
 [4.9, 3.1, 1.5, 0.1],
 [5.4, 3.7, 1.5, 0.2],
 [4.8, 3.4, 1.6, 0.2],
 [4.8, 3.0, 1.4, 0.1],
 [4.3, 3.0, 1.1, 0.1],
 [5.8, 4.0, 1.2, 0.2],
 [5.7, 4.4, 1.5, 0.4],
 [5.4, 3.9, 1.3, 0.4],
 [5.1, 3.5, 1.4, 0.3],
 [5.7, 3.8, 1.7, 0.3],
 [5.1, 3.8, 1.5, 0.3],
 [5.4, 3.4, 1.7, 0.2],
 [5.1, 3.7, 1.5, 0.4],
 [4.6, 3.6, 1.0, 0.2],
 [5.1, 3.3, 1.7, 0.5],
 [4.8, 3.4, 1.9, 0.2],
 [5.0, 3.0, 1.6, 0.2],
 [5.0, 3.4, 1.6, 0.4],
 [5.2, 3.5, 1.5, 0.2],
 [5.2, 3.4, 1.4, 0.2],
 [4.7, 3.2, 1.6, 0.2],
 [4.8, 3.1, 1.6, 0.2],
 [5.4, 3.4, 1.5, 0.4],
 [5.2, 4.1, 1.5, 0.1],
 [5.5, 4.2, 1.4, 0.2],
 [4.9, 3.1, 1.5, 0.1],
 [5.0, 3.2, 1.2, 0.2],
 [5.5, 3.5, 1.3, 0.2],
 [4.9, 3.1, 1.5, 0.1],
 [4.4, 3.0, 1.3, 0.2],
 [5.1, 3.4, 1.5, 0.2],
 [5.0, 3.5, 1.3, 0.3],
 [4.5, 2.3, 1.3, 0.3],
 [4.4, 3.2, 1.3, 0.2],
 [5.0, 3.5,

In [12]:
predicted = get_predictions(example_inputs).json()["output"]

In [13]:
iris["predicted"] = predicted

In [14]:
iris

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Label,predicted
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,0
2,4.7,3.2,1.3,0.2,Iris-setosa,0
3,4.6,3.1,1.5,0.2,Iris-setosa,0
4,5.0,3.6,1.4,0.2,Iris-setosa,0
5,5.4,3.9,1.7,0.4,Iris-setosa,0
6,4.6,3.4,1.4,0.3,Iris-setosa,0
7,5.0,3.4,1.5,0.2,Iris-setosa,0
8,4.4,2.9,1.4,0.2,Iris-setosa,0
9,4.9,3.1,1.5,0.1,Iris-setosa,0


### Stop All Serving Containers

Finally, we will shut down the serving container we launched for the test.

In [ ]:
!docker kill $(docker ps -q)